####  Холодные звонки

Классификатор только на основе телефонного номера. Без интернет логов.

In [2]:
#Config
from pyspark import SparkConf, SparkContext, HiveContext
import re
import numpy as np
import pandas as pd
import datetime
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.feature import HashingTF
from pyspark.mllib.classification import LogisticRegressionWithSGD, NaiveBayes, NaiveBayesModel
import scipy.sparse as sps
from pyspark.mllib.linalg import Vectors
import datetime
sc.stop()
conf = SparkConf().set("spark.executor.instances", 64).set("spark.driver.maxResultSize", "32g")
sc = SparkContext(conf=conf)
hc = HiveContext(sc)



In [6]:
create_table_query = '''
create table user_kposminin.ccalls_sig_2 as
select 
  c.ymd,
  c.phone_num,
  c.approve, 
  c.full_app, 
  c.stop_cond, 
  if(c.status in ('Успешно','Уже оставил заявку','Действующий клиент'),1,0) as target,
  if(c.status in ('Нет потребности в кредите','Не требуется сейчас','Попросил не звонить/Претензионный'),1,0) as no_need,
  if(c.status in ('Недозвон','Не отвечает','Левый контакт','Занято','Автоответчик','Недоступен','Ошибка телефонии',
                'Сбрасывает','Клиент умер','Звонок сорвался',''),1,0) as not_started,
  c.util_30,           
  c.status, 
  from_unixtime(unix_timestamp(dt), 'u') as weekday,
  hour(dt) as raw_hour,
  trim(pc.provider) as mob_provider,
  r.ind,
  r.pop_country_share, 
  r.pop_city_share, 
  r.population / r.area_sq_km as density,
  r.area_sq_km,
  trim(r.federal_district) as federal_district, 
  r.avg_salary_2015_rub, 
  r.utc_time_zone_val
from
  user_kposminin.cold_calls_4 c
  left join hermes.phone_codes pc on trim(pc.phone_code) = substr(c.phone_num,2,9)
  left join (select trim(pc_name) as pc_name, max(reg_stat_ind) as reg_stat_ind from user_kposminin.tmp_pc_2 group by  trim(pc_name)) tmp on tmp.pc_name = trim(pc.region)
  left join hermes.region_stat r on r.ind = tmp.reg_stat_ind
;
'''

In [9]:
read_query = '''
select * from user_kposminin.ccalls_sig_2
where not ind is Null
'''


data = hc.sql(read_query) 

train_data = data \
    .filter("ymd < '2016-06-15' and rand() * 7 < 1 ") \
    .toPandas()

test_data = data \
    .filter("ymd >= '2016-06-15' and rand() * 2 < 1 ") \
    .toPandas()

In [19]:
train_data = train_data[~train_data['ind'].isnull()]
test_data = test_data[~test_data['ind'].isnull()]

In [36]:
#train_data.columns
train_data.columns[9:]

Index([           u'status',           u'weekday',          u'raw_hour',
            u'mob_provider',               u'ind', u'pop_country_share',
          u'pop_city_share',           u'density',        u'area_sq_km',
        u'federal_district', 
       ...
                u'ind_78.0',          u'ind_73.0',          u'ind_35.0',
                u'ind_61.0',          u'ind_84.0',          u'ind_83.0',
                u'ind_55.0',          u'ind_47.0',          u'ind_33.0',
                u'ind_26.0'],
      dtype='object', length=113)

In [33]:
for col in ['mob_provider','federal_district']:
    for c in pd.concat([train_data,test_data])[col].dropna().unique():
        train_data[col + '_' + c] = train_data[col].map(lambda v: 1 * (c == v))
        test_data[col + '_' + c]  =  test_data[col].map(lambda v: 1 * (c == v))

for col in ['ind']:
    for c in pd.concat([train_data,test_data])[col].dropna().unique():
        train_data[col + '_' + str(c)] = train_data[col].map(lambda v: 1 * (c == v))
        test_data[col + '_' + str(c)]  =  test_data[col].map(lambda v: 1 * (c == v))


/opt/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [67]:
#train_data.columns
#feat_cols = [u'weekday',u'raw_hour', u'pop_country_share',u'pop_city_share', u'density', u'area_sq_km',
#       u'avg_salary_2015_rub', u'utc_time_zone_val']
feat_cols = [c for c in train_data.columns if not c in [u'ymd',u'phone_num',u'approve',u'full_app',u'stop_cond',
                u'target',u'no_need', u'not_started',u'util_30',u'status',u'mob_provider','federal_district','ind']]
label_col = 'approve'

In [68]:
label_col

'approve'

In [65]:
test_data[label_col + '_XGBoost']= [e[1] for e in clfXGB.predict_proba(Xt)]
train_data[label_col + '_XGBoost']= [e[1] for e in clfXGB.predict_proba(X)]

/opt/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from IPython.kernel.zmq import kernelapp as app


In [69]:
from sklearn.preprocessing import Imputer
imp = Imputer(missing_values='NaN', strategy='mean')
X, y  = imp.fit_transform(train_data[feat_cols]), train_data[label_col]
Xt,yt = imp.transform(test_data[feat_cols])     , test_data[label_col]


In [71]:
import sklearn
import xgboost as xgb

clfXGB = xgb.XGBClassifier()
clfXGB.fit(X,y)
print('{0} AUCROC test: {1}. train {2}'.format(
               'XGBoost',
                sklearn.metrics.roc_auc_score(
                  y_true = yt, 
                  y_score = [e[1] for e in clfXGB.predict_proba(Xt)]
                ),
                sklearn.metrics.roc_auc_score(
                  y_true = y, 
                  y_score = [e[1] for e in clfXGB.predict_proba(X)]
                )                                              
))

XGBoost AUCROC test: 0.52493187505. train 0.551681911162


In [73]:
from sklearn.ensemble import AdaBoostClassifier

clfAB = AdaBoostClassifier(learning_rate = 0.1, n_estimators = 150)
clfAB.fit(X,y)
print('{0} AUCROC: {1}'.format('AdaBoostClassifier',sklearn.metrics.roc_auc_score(
                y_true = yt, 
                y_score = [e[1] for e in clfAB.predict_proba(Xt)]
    )))

AdaBoostClassifier AUCROC: 0.52457212172


In [75]:
import sklearn.linear_model
mLR = sklearn.linear_model.LogisticRegression(penalty = 'l1',C =1, class_weight = 'auto')
mLR.fit(X,y)
print('{0} AUCROC: {1}'.format('LogisticRegression',sklearn.metrics.roc_auc_score(
                y_true = yt, 
                y_score = [e[1] for e in mLR.predict_proba(Xt)]
    )))

LogisticRegression AUCROC: 0.524069080925


/opt/anaconda/lib/python2.7/site-packages/sklearn/utils/class_weight.py:65: DeprecationWarning: The class_weight='auto' heuristic is deprecated in 0.17 in favor of a new heuristic class_weight='balanced'. 'auto' will be removed in 0.19
  " 0.19", DeprecationWarning)


In [74]:
from sklearn.ensemble import AdaBoostClassifier

clfAB = AdaBoostClassifier(learning_rate = 0.1, n_estimators = 250)
clfAB.fit(X,y)
print('{0} AUCROC: {1}'.format('AdaBoostClassifier',sklearn.metrics.roc_auc_score(
                y_true = yt, 
                y_score = [e[1] for e in clfAB.predict_proba(Xt)]
    )))

AdaBoostClassifier AUCROC: 0.525027464328
